In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [2]:
df_sale = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
df_sale_eval = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
df_calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
df_price = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

In [3]:
date_columns = df_sale.columns[df_sale.columns.str.contains("d_")]
print(date_columns)

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1913)


In [4]:
dates_s = [pd.to_datetime(df_calendar.loc[df_calendar['d']==str_date,'date'].values[0]) for str_date in date_columns]
print(dates_s)

[Timestamp('2011-01-29 00:00:00'), Timestamp('2011-01-30 00:00:00'), Timestamp('2011-01-31 00:00:00'), Timestamp('2011-02-01 00:00:00'), Timestamp('2011-02-02 00:00:00'), Timestamp('2011-02-03 00:00:00'), Timestamp('2011-02-04 00:00:00'), Timestamp('2011-02-05 00:00:00'), Timestamp('2011-02-06 00:00:00'), Timestamp('2011-02-07 00:00:00'), Timestamp('2011-02-08 00:00:00'), Timestamp('2011-02-09 00:00:00'), Timestamp('2011-02-10 00:00:00'), Timestamp('2011-02-11 00:00:00'), Timestamp('2011-02-12 00:00:00'), Timestamp('2011-02-13 00:00:00'), Timestamp('2011-02-14 00:00:00'), Timestamp('2011-02-15 00:00:00'), Timestamp('2011-02-16 00:00:00'), Timestamp('2011-02-17 00:00:00'), Timestamp('2011-02-18 00:00:00'), Timestamp('2011-02-19 00:00:00'), Timestamp('2011-02-20 00:00:00'), Timestamp('2011-02-21 00:00:00'), Timestamp('2011-02-22 00:00:00'), Timestamp('2011-02-23 00:00:00'), Timestamp('2011-02-24 00:00:00'), Timestamp('2011-02-25 00:00:00'), Timestamp('2011-02-26 00:00:00'), Timestamp('20

In [5]:
df_ev_1 = pd.DataFrame({'holiday':'Event 1','ds':df_calendar[~df_calendar['event_name_1'].isna()]['date']})
print(df_ev_1)

      holiday          ds
8     Event 1  2011-02-06
16    Event 1  2011-02-14
23    Event 1  2011-02-21
39    Event 1  2011-03-09
46    Event 1  2011-03-16
...       ...         ...
1926  Event 1  2016-05-08
1948  Event 1  2016-05-30
1951  Event 1  2016-06-02
1956  Event 1  2016-06-07
1968  Event 1  2016-06-19

[162 rows x 2 columns]


In [6]:
df_ev_2 = pd.DataFrame({'holiday': 'Event 2', 'ds': df_calendar[~df_calendar['event_name_2'].isna()]['date']})
print(df_ev_2)

      holiday          ds
85    Event 2  2011-04-24
827   Event 2  2013-05-05
1177  Event 2  2014-04-20
1233  Event 2  2014-06-15
1968  Event 2  2016-06-19


In [7]:
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': df_calendar[df_calendar['snap_CA'] == 1]['date']})
print(df_ev_3)

      holiday          ds
3     snap_CA  2011-02-01
4     snap_CA  2011-02-02
5     snap_CA  2011-02-03
6     snap_CA  2011-02-04
7     snap_CA  2011-02-05
...       ...         ...
1955  snap_CA  2016-06-06
1956  snap_CA  2016-06-07
1957  snap_CA  2016-06-08
1958  snap_CA  2016-06-09
1959  snap_CA  2016-06-10

[650 rows x 2 columns]


In [8]:
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': df_calendar[df_calendar['snap_TX'] == 1]['date']})
print(df_ev_4)

      holiday          ds
3     snap_TX  2011-02-01
5     snap_TX  2011-02-03
7     snap_TX  2011-02-05
8     snap_TX  2011-02-06
9     snap_TX  2011-02-07
...       ...         ...
1958  snap_TX  2016-06-09
1960  snap_TX  2016-06-11
1961  snap_TX  2016-06-12
1962  snap_TX  2016-06-13
1964  snap_TX  2016-06-15

[650 rows x 2 columns]


In [9]:
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': df_calendar[df_calendar['snap_WI'] == 1]['date']})
print(df_ev_5)

      holiday          ds
4     snap_WI  2011-02-02
5     snap_WI  2011-02-03
7     snap_WI  2011-02-05
8     snap_WI  2011-02-06
10    snap_WI  2011-02-08
...       ...         ...
1958  snap_WI  2016-06-09
1960  snap_WI  2016-06-11
1961  snap_WI  2016-06-12
1963  snap_WI  2016-06-14
1964  snap_WI  2016-06-15

[650 rows x 2 columns]


In [10]:
holidays = pd.concat((df_ev_1, df_ev_2, df_ev_3, df_ev_4, df_ev_5))
print(holidays)

      holiday          ds
8     Event 1  2011-02-06
16    Event 1  2011-02-14
23    Event 1  2011-02-21
39    Event 1  2011-03-09
46    Event 1  2011-03-16
...       ...         ...
1958  snap_WI  2016-06-09
1960  snap_WI  2016-06-11
1961  snap_WI  2016-06-12
1963  snap_WI  2016-06-14
1964  snap_WI  2016-06-15

[2117 rows x 2 columns]


In [11]:
holidays.tail(40)

,holiday,ds
1859,snap_WI,2016-03-02
1860,snap_WI,2016-03-03
1862,snap_WI,2016-03-05
1863,snap_WI,2016-03-06
1865,snap_WI,2016-03-08
1866,snap_WI,2016-03-09
1868,snap_WI,2016-03-11
1869,snap_WI,2016-03-12
1871,snap_WI,2016-03-14
1872,snap_WI,2016-03-15


In [12]:
df_sale_group_item = df_sale[np.hstack([['dept_id','store_id'],date_columns])].groupby(['dept_id','store_id']).sum()
df_sale_group_item = df_sale_group_item.reset_index()

In [14]:
from fbprophet import Prophet
from multiprocessing import Pool,cpu_count

In [13]:
df_sale_group_item

,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_1,CA_1,297,284,214,175,182,191,224,263,...,293,317,268,302,237,287,297,331,419,329
1,FOODS_1,CA_2,406,408,238,240,220,263,304,353,...,535,504,515,327,366,422,334,579,623,520
2,FOODS_1,CA_3,341,326,260,231,220,258,285,309,...,420,500,457,301,331,306,276,335,510,423
3,FOODS_1,CA_4,113,124,75,97,107,84,100,111,...,284,241,193,196,185,201,190,229,250,257
4,FOODS_1,TX_1,152,154,109,126,108,153,72,126,...,204,244,245,193,138,176,198,201,218,221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,HOUSEHOLD_2,TX_2,199,183,117,108,98,110,133,200,...,210,246,270,134,193,182,187,211,260,244
66,HOUSEHOLD_2,TX_3,120,138,97,83,46,102,120,141,...,198,218,201,127,189,158,161,131,185,191
67,HOUSEHOLD_2,WI_1,148,151,66,47,0,105,126,186,...,196,225,199,143,131,158,169,192,203,224
68,HOUSEHOLD_2,WI_2,98,94,69,80,32,107,100,111,...,136,138,137,139,147,113,127,160,194,177


In [15]:
ids = []
for i in range(0,df_sale_group_item.shape[0]):
    ids = ids + [(df_sale_group_item[i:i+1]['dept_id'].values[0],df_sale_group_item[i:i+1]['store_id'].values[0])]

In [18]:
print(len(ids))

70


In [27]:
def CreateTimeSeries(dept_id, store_id):
    item_series =  df_sale_group_item[(df_sale_group_item.dept_id == dept_id) & (df_sale_group_item.store_id == store_id)]
    dates = pd.DataFrame({'ds': dates_s}, index=range(len(dates_s)))
    dates['y'] = item_series[date_columns].values[0].transpose()     
    return dates

def run_prophet(dept_id, store_id):
    timeserie = CreateTimeSeries(dept_id, store_id)
    model = Prophet(holidays = holidays, uncertainty_samples = False, n_changepoints = 50, changepoint_range = 0.8, changepoint_prior_scale = 0.7, seasonality_mode = 'multiplicative')
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(forecast)
    return np.append(np.array([dept_id,store_id]),forecast['yhat'].values.transpose())


In [33]:
with Pool(cpu_count()) as p:
    predictions = list(p.starmap(run_prophet,ids))
    

In [31]:
print(predictions)

[array(['FOODS_1', 'CA_1', '245.45887292816803', '238.3091103942481',
       '239.821615549954', '252.20395895451514', '306.5946620715991',
       '351.11147382008005', '309.12505988097536', '253.6688606867547',
       '252.2631095658372', '249.26981296723818', '255.9825123727037',
       '324.05022016979717', '380.19902770730715', '319.24606511991976',
       '270.73520142654354', '259.5961372421839', '267.89259518832455',
       '281.3667396630007', '334.3371367543051', '389.606031358845',
       '347.4947386517814', '275.95590343286045', '270.23803899626563',
       '272.70552140171077', '285.5639909170914', '339.62777752990576',
       '396.3510048129405', '349.8582574579111'], dtype='<U32'), array(['FOODS_1', 'CA_2', '349.98551159657336', '347.1297163877523',
       '361.796731960584', '385.3374790784237', '446.93384843424263',
       '571.611502908394', '507.3943140784706', '357.03225510771705',
       '360.68942939493735', '367.9523464649329', '398.6058828869924',
       '466.57

In [35]:
df_sub_val = pd.DataFrame()
for k in range(0,len(predictions)):
    dept_id = predictions[k][0]
    store_id = predictions[k][1]
    
    df_item = df_sale.loc[(df_sale.dept_id == dept_id) &(df_sale.store_id==store_id)][['id']]
    df_item['val'] = df_sale[(df_sale.dept_id == dept_id) & (df_sale.store_id == store_id)].iloc[:, np.r_[0,-28:0]].sum(axis = 1)
    for i in range(1,29):
        df_item[f'F{i}'] = (df_item['val'] * float(predictions[k][i+1]) / df_item['val'].sum())
    df_sub_val = pd.concat([df_sub_val, df_item])

df_sub_val = df_sub_val.drop('val',axis=1)

In [38]:
df_sub_val.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
1612,FOODS_1_001_CA_1_validation,1.010371,0.980941,0.987166,1.038135,1.262021,1.445264,1.272437,1.044165,1.038379,...,1.376216,1.603717,1.430376,1.135904,1.112368,1.122525,1.175454,1.397994,1.631481,1.440105
1613,FOODS_1_002_CA_1_validation,0.367408,0.356706,0.358970,0.377504,0.458917,0.525550,0.462704,0.379696,0.377592,...,0.500442,0.583170,0.520137,0.413056,0.404498,0.408191,0.427438,0.508361,0.593266,0.523675
1614,FOODS_1_003_CA_1_validation,0.673581,0.653960,0.658111,0.692090,0.841347,0.963509,0.848291,0.696110,0.692253,...,0.917477,1.069145,0.953584,0.757270,0.741579,0.748350,0.783636,0.931996,1.087654,0.960070
1615,FOODS_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1616,FOODS_1_005_CA_1_validation,1.040988,1.010666,1.017081,1.069594,1.300264,1.489060,1.310996,1.075807,1.069845,...,1.417920,1.652314,1.473721,1.170326,1.146076,1.156541,1.211073,1.440357,1.680920,1.483745


In [39]:
#Evaluation

In [40]:
df_sale = df_sale_eval
date_columns = df_sale.columns[df_sale.columns.str.contains("d_")]
dates_s = [pd.to_datetime(df_calendar.loc[df_calendar['d'] == str_date,'date'].values[0]) for str_date in date_columns]

df_ev_1 = pd.DataFrame({'holiday': 'Event 1', 'ds': df_calendar[~df_calendar['event_name_1'].isna()]['date']})
df_ev_2 = pd.DataFrame({'holiday': 'Event 2', 'ds': df_calendar[~df_calendar['event_name_2'].isna()]['date']})
df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': df_calendar[df_calendar['snap_CA'] == 1]['date']})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': df_calendar[df_calendar['snap_TX'] == 1]['date']})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': df_calendar[df_calendar['snap_WI'] == 1]['date']})
holidays = pd.concat((df_ev_1, df_ev_2, df_ev_3, df_ev_4, df_ev_5))

df_sale_group_item = df_sale[np.hstack([['dept_id','store_id'],date_columns])].groupby(['dept_id','store_id']).sum()
df_sale_group_item = df_sale_group_item.reset_index()

In [41]:
ids = []
for i in range(0,df_sale_group_item.shape[0]):
    ids = ids + [(df_sale_group_item[i:i+1]['dept_id'].values[0],df_sale_group_item[i:i+1]['store_id'].values[0])]

print(f'Parallelism on {cpu_count()} CPU')
with Pool(cpu_count()) as p:
    predictions  = list(p.starmap(run_prophet, ids))

Parallelism on 4 CPU


In [42]:
df_sub_eval = pd.DataFrame()
for k in range(0, len(predictions)):
    dept_id = predictions[k][0]
    store_id = predictions[k][1]

    df_item = df_sale.loc[(df_sale.dept_id == dept_id) & (df_sale.store_id == store_id)][['id']]
    df_item['val'] = df_sale[(df_sale.dept_id == dept_id) & (df_sale.store_id == store_id)].iloc[:, np.r_[0,-28:0]].sum(axis = 1)
    for i in range(1,29):
        df_item[f'F{i}'] = (df_item['val'] * float(predictions[k][i+1]) / df_item['val'].sum())
    df_sub_eval = pd.concat([df_sub_eval, df_item])

df_sub_eval = df_sub_eval.drop('val',axis=1)

In [45]:
df_sub_eval.id.describe()

count                               30490
unique                              30490
top       HOUSEHOLD_2_348_CA_4_evaluation
freq                                    1
Name: id, dtype: object

In [46]:
df_sub = pd.concat([df_sub_val,df_sub_eval], sort=False)
df_sub = df_sub.sort_values('id').reset_index(drop = True)

df_sub.to_csv('submission_prophet.csv', index=False)
df_sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.716333,0.701238,0.705431,0.735355,0.868339,1.007966,0.891277,0.698589,0.713352,...,0.889369,1.038459,0.919916,0.751312,0.725439,0.737329,0.748736,0.876336,1.010169,0.870906
1,FOODS_1_001_CA_1_validation,1.010371,0.980941,0.987166,1.038135,1.262021,1.445264,1.272437,1.044165,1.038379,...,1.376216,1.603717,1.430376,1.135904,1.112368,1.122525,1.175454,1.397994,1.631481,1.440105
2,FOODS_1_001_CA_2_evaluation,0.561536,0.564227,0.590488,0.628085,0.716184,0.891347,0.803969,0.605313,0.609523,...,0.778171,0.957018,0.861150,0.649694,0.641955,0.667480,0.685544,0.766968,0.937557,0.900290
3,FOODS_1_001_CA_2_validation,0.869564,0.862469,0.898910,0.957399,1.110439,1.420210,1.260658,0.887072,0.896159,...,1.195436,1.528706,1.371656,0.967828,0.974064,1.024656,1.097624,1.266936,1.606554,1.440235
4,FOODS_1_001_CA_3_evaluation,0.847563,0.830483,0.840333,0.861481,0.913460,1.114295,1.079937,0.871380,0.859204,...,0.956091,1.177055,1.134752,0.904505,0.872155,0.896324,0.865413,0.906612,1.096569,1.208563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_WI_1_validation,0.029500,0.028486,0.029428,0.029632,0.035832,0.041434,0.036108,0.029481,0.028661,...,0.034993,0.043783,0.038192,0.028157,0.027291,0.028408,0.028807,0.035235,0.044120,0.038456
60976,HOUSEHOLD_2_516_WI_2_evaluation,0.061469,0.060068,0.061547,0.065262,0.075017,0.084919,0.075363,0.056740,0.061908,...,0.083111,0.089491,0.079537,0.067869,0.063177,0.066289,0.068769,0.078160,0.087639,0.058285
60977,HOUSEHOLD_2_516_WI_2_validation,0.030717,0.029993,0.030727,0.032587,0.037571,0.039345,0.037576,0.032999,0.033175,...,0.037923,0.041113,0.036919,0.030022,0.029071,0.029614,0.031321,0.036221,0.041044,0.035995
60978,HOUSEHOLD_2_516_WI_3_evaluation,0.063589,0.060262,0.060267,0.063569,0.078687,0.089522,0.084109,0.060271,0.062393,...,0.083601,0.093380,0.087519,0.068289,0.063458,0.064324,0.065543,0.080328,0.090774,0.082571
